In [1]:
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')

In [2]:
mnist_train = np.load('data/mnist_train.npy')
X_train = np.load('data/X_train.npy')
y_train = np.load('data/y_train.npy')
mnist_test = np.load('data/mnist_test.npy')
X_test = np.load('data/X_test.npy')
y_test = np.load('data/y_test.npy')

In [27]:
X_p, X_var = np.load('predictions/X_test.npy')
print(X_p.shape)
print(X_var.shape)
X_std = np.sqrt(X_var)

(10000, 10)
(10000, 10)


In [7]:
predictions = np.argmax(X_p, axis=-1)

In [9]:
accuracy = np.sum(predictions == y_test) / len(y_test)
print(accuracy)

0.9928


In [36]:
arg_sorted_p = np.argsort(X_p, axis=-1)
print(arg_sorted_p[0])
print(X_p[0])
print(X_std[0])
print(X_p[0][arg_sorted_p[0]])

highest_arg = arg_sorted_p[:, -1]
second_highest_arg = arg_sorted_p[:, -2]

highest_p = X_p[np.arange(len(X_p)), highest_arg]
second_highest_p  = X_p[np.arange(len(X_p)), second_highest_arg]
highest_p_std = X_std[np.arange(len(X_p)), highest_arg]

def R_acc(alpha=-1, beta=0):
    eps = 1
    reject_img = highest_p - second_highest_p < eps * highest_p_std
    print("Rejecting: {}".format(np.sum(reject_img)))
    alpha * np.sum(predictions == y_test)
    correct_accept = (y_test == predictions) & np.logical_not(reject_img)
    correct_reject = (y_test == predictions) & reject_img
    return np.sum(predictions == y_test & np.logical_not(reject_img)) + alpha * np.sum(predictions == y_test & np.logical_not(reject_img))

print(R_acc())

[4 6 8 5 2 0 1 9 3 7]
[1.11111543e-04 1.11113623e-04 1.11111118e-04 2.10415329e-04
 1.11111111e-04 1.11111113e-04 1.11111111e-04 9.98101360e-01
 1.11111111e-04 1.11622929e-04]
[0.01054036 0.01054046 0.01054034 0.01450417 0.01054034 0.01054034
 0.01054034 0.04353201 0.01054034 0.01056459]
[1.11111111e-04 1.11111111e-04 1.11111111e-04 1.11111113e-04
 1.11111118e-04 1.11111543e-04 1.11113623e-04 1.11622929e-04
 2.10415329e-04 9.98101360e-01]
Rejecting: 61
None
